In [6]:
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re
import ast
import csv
from gensim.models.word2vec import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from sklearn import cluster
from sklearn import metrics

musinsa = pd.read_csv("musinsa_data_man.csv")

a = musinsa['top_tag']
b = musinsa['bottom_tag']


sentence = []
for idx, i in enumerate(a):
    sentence.append(a.loc[idx] + ', ' + b.loc[idx])

musinsa['total_tag'] = sentence

# 단어들의 벡터화
model = Word2Vec(sentence, min_count=1, iter=200, size=300, sg=1)
model.init_sims(replace=True)
# print("섹시과 관련된 키워드 : ", model.most_similar("섹시"))
# print("스트리트와 관련된 키워드 : ", model.most_similar("스트리트"))
# print("심플와 관련된 키워드 : ", model.most_similar("심플"))


# ## 학습된 모델 사용시 벡터의 평균 점수 사용
# •밑의 함수에 대한 요약 해석
# : 한 사진마다 태그가 없으면 벡터화할 단어가 없으므로 0 값
# : 단어가 있다면 태그 하나당 수치화(벡터화)의 수준을 300개로 할당
#
# -- ex) crop에 대한 점수(벡터)- 300개
#
# •결론: 각 단어마다 300개의 수치화된 점수로 구성된다.
#


def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
#     항상 generate_missing if 안으로 들어감.
#     각 단어들이 만든 모델에 있으면 vectorized 변수에 그 정보를 저장
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

sentence = pd.DataFrame(sentence, columns=None)


def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = musinsa['total_tag'].apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
    return list(embeddings)

training_embeddings = get_word2vec_embeddings(model, musinsa, generate_missing=True)


# 모든 벡터에 대한 단어 저장
vocabs = model.wv.vocab.keys()
# print(vocabs)

# 우리의 태그들에 대한 벡터
word_vectors_list= [model[v] for v in vocabs]
# print(word_vectors_list)

# 태그들을 벡터화 시킨 값을 x에 저장
# 2개의 성분으로 벡터들을 축소
pca = PCA(n_components=2)
X = pca.fit_transform(training_embeddings)

# kmeans 클러스터링을 이용해 군집
NUM_CLUSTERS=5
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(X)

labels = kmeans.labels_
centroids = kmeans.cluster_centers_

musinsa['class'] = labels

musinsa.to_csv('clustering_style.csv',mode='w',index=False,encoding='utf-8')
print("finish!!!!!!!!!!")

Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
c:\users\mks10\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
c:\users\mks10\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\mks10\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


finish!!!!!!!!!!
